In [1]:
# %% Impor Library
import numpy as np
import matplotlib.pyplot as plt

/home/mthudaa/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [2]:
# %% Fungsi Baca Data
def baca_data(file_path, start_time, end_time, bit=10):
    """Membaca file data dan return waktu dan nilai digital"""
    data = np.loadtxt(file_path)
    time = data[:, 0]
    vcp = data[:, 4]  # Ambil kolom VCP (kolom ke-5)
    vcn = data[:, 5]  # Ambil kolom VCN (kolom ke-6)
    ck = data[:, 1]  # Ambil kolom CKO (kolom ke-2)
    ck_bits = (ck >= 0.9).astype(int)  # Ambil bit CKO
    vc = vcp - vcn  # Hitung Vc dari VCP dan VCN

    # Pilih data dalam window waktu
    mask = (time >= start_time) & (time <= end_time)
    time_sel = time[mask]
    time_sel = time_sel - time_sel[0]  # Normalisasi waktu
    vc_sel = vc[mask]
    ck_sel = ck_bits[mask]

    ck_points = np.zeros(2**bit, dtype=int)  # Tempat untuk menyimpan titik CKO
    already = 0
    for i in range(1, len(ck_sel)-1):
        if ck_sel[i] == 0 and ck_sel[i+1] == 1 and already < 2**bit:
            ck_points[already] = i + 1300  # Ambil titik CKO
            already += 1
        elif already >= 2**bit:
            break

    # Take samples at every n points
    vc_sel_sampled = vc_sel[ck_points.astype(int)]
    time_sel_sampled = time_sel[ck_points.astype(int)]

    return time_sel_sampled, vc_sel_sampled

def compute_fft(time, signal):
    """Compute the FFT of a signal with windowing."""
    n = len(signal)
    # Apply Hanning window
    window = np.blackman(n)
    windowed_signal = signal * window
    
    fft_result = np.fft.fft(windowed_signal)
    fft_result = fft_result[:n//2]  # Take only the positive frequencies
    sampling_rate = 1 / (time[1] - time[0])  # Calculate sampling rate from time
    freqs = np.fft.fftfreq(len(signal), d=1/sampling_rate)[:n//2]
    power_spectrum = 20 * np.log10(np.abs(fft_result))
    power_ref = np.max(power_spectrum)
    mag = power_spectrum - power_ref  # Normalize to dB
    return freqs, mag

In [ ]:
# %% Parameter Input
# File and ADC Configuration
file_path = "sim/th_tb.txt"    # Data file path
bit = 8                            # ADC resolution (bits)
nbpt = 2**bit                      # Number of FFT points (power of 2)

# Timing Parameters
start_time = 0.6e-6               # Start time for analysis (seconds)
sampling_period = 0.4e-6          # Sampling period (seconds)
fs_param = 1/sampling_period      # Sampling frequency (Hz)
duration = nbpt/fs_param          # Signal duration (seconds)
end_time = start_time + duration  # End time for analysis (seconds)

# Frequency Analysis Parameters
fin_ratio = 0.1                   # Input frequency ratio to sampling frequency
fin = (round(fin_ratio * nbpt)/nbpt) * fs_param  # Input frequency (Hz)
num_harmonics = 10                # Number of harmonics to analyze

# Print Parameter Values
print("=== Configuration Parameters ===")
print(f"File Path: {file_path}")
print(f"ADC Resolution: {bit} bits")
print(f"Number of FFT Points: {nbpt}")
print("\n=== Timing Parameters ===")
print(f"Start Time: {start_time*1e6:.2f} µs")
print(f"Sampling Period: {sampling_period*1e6:.2f} µs")
print(f"Sampling Frequency: {fs_param/1e3:.2f} kHz")
print(f"Duration: {duration*1e3:.2f} ms")
print(f"End Time: {end_time*1e6:.2f} µs")
print("\n=== Frequency Analysis Parameters ===")
print(f"Input Frequency Ratio: {fin_ratio}")
print(f"Input Frequency: {fin/1e3:.2f} kHz")
print(f"Number of Harmonics: {num_harmonics}")

=== Configuration Parameters ===
File Path: th_tb.txt
ADC Resolution: 8 bits
Number of FFT Points: 256

=== Timing Parameters ===
Start Time: 0.60 µs
Sampling Period: 0.40 µs
Sampling Frequency: 2500.00 kHz
Duration: 0.10 ms
End Time: 103.00 µs

=== Frequency Analysis Parameters ===
Input Frequency Ratio: 0.1
Input Frequency: 253.91 kHz
Number of Harmonics: 10


In [ ]:
time, vc = baca_data(file_path, start_time, end_time, bit)
time.shape, vc.shape